In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.13.1
Torchvision Version:  0.14.1


In [2]:
data_dir = "/media/abdul/8266fdc5-61f6-47d6-9e82-aa0e56f4b319/Ilyas/Gender/CINIC"


num_classes = 10
batch_size = 32
num_epochs = 10

model = models.mobilenet_v2(pretrained=True)
# Update model output size to 2.
model.classifier = nn.Linear(1280, num_classes)
model

/home/abdul/miniconda3/envs/detr/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/abdul/miniconda3/envs/detr/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [3]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
       # print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
       
    ]),
    'val': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [5]:
train_dataset_size = len(image_datasets['train'])
val_dataset_size = len(image_datasets['val'])

print("Number of training samples: ", train_dataset_size)
print("Number of validation samples: ", val_dataset_size)


Number of training samples:  90000
Number of validation samples:  90000


In [6]:


model_ft = model.to(device)

params_to_update = []

for name, param in model_ft.named_parameters():
    param.requires_grad == False
    

for name,param in model_ft.named_parameters():

    if 'features.17' in name:
        param.requires_grad == True
        params_to_update.append(param)        

    if 'features.18' in name:
        param.requires_grad == True
        params_to_update.append(param)
        
    if 'classifier' in name:
        param.requires_grad == True
        params_to_update.append(param)
        



In [7]:
optimizer_ft = optim.SGD(params_to_update, lr=0.0005, momentum=0.9)

criterion = nn.CrossEntropyLoss()

model_ft, hist = train_model(model_ft, 
                             dataloaders_dict, 
                             criterion, 
                             optimizer_ft, 
                             num_epochs=25)

----------
train Loss: 1.7523 Acc: 0.3660
val Loss: 1.6496 Acc: 0.4157

----------
train Loss: 1.6515 Acc: 0.4032
val Loss: 1.6115 Acc: 0.4272

----------
train Loss: 1.6193 Acc: 0.4165
val Loss: 1.5787 Acc: 0.4344

----------
train Loss: 1.5947 Acc: 0.4259
val Loss: 1.5253 Acc: 0.4508

----------
train Loss: 1.5810 Acc: 0.4309
val Loss: 1.5284 Acc: 0.4510

----------
train Loss: 1.5656 Acc: 0.4336
val Loss: 1.5427 Acc: 0.4471

----------
train Loss: 1.5567 Acc: 0.4396
val Loss: 1.4937 Acc: 0.4636

----------
train Loss: 1.5489 Acc: 0.4430
val Loss: 1.4850 Acc: 0.4654

----------
train Loss: 1.5378 Acc: 0.4443
val Loss: 1.4835 Acc: 0.4674

----------
train Loss: 1.5299 Acc: 0.4496
val Loss: 1.4925 Acc: 0.4639

----------
train Loss: 1.5188 Acc: 0.4557
val Loss: 1.4860 Acc: 0.4685

----------
train Loss: 1.5153 Acc: 0.4559
val Loss: 1.4673 Acc: 0.4750

----------
train Loss: 1.5071 Acc: 0.4575
val Loss: 1.5119 Acc: 0.4646

----------
train Loss: 1.4992 Acc: 0.4616
val Loss: 1.4880 Acc: 

In [8]:
train_dataset_size = len(image_datasets['train'])
val_dataset_size = len(image_datasets['val'])

print("Number of training samples: ", train_dataset_size)
print("Number of validation samples: ", val_dataset_size)
img_test_transforms = transforms.Compose([
    transforms.Resize((32,32)),
        transforms.ToTensor(),
    ])
test_data = torchvision.datasets.ImageFolder(data_dir + '/test',transform=img_test_transforms)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          shuffle=False, num_workers=4)


Number of training samples:  90000
Number of validation samples:  90000


In [ ]:
model_ft.eval()  # it-disables-dropout
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
       
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_ft(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        print("input",labels)
        print("output",predicted)
        
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model: {} %'.format(100 * correct / total))
    print('correct: {:d}  total: {:d}'.format(correct, total))


input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 4, 0, 2, 0, 0, 6, 8, 2, 0, 0, 1, 0, 0, 8, 0, 8, 8, 0, 0, 9, 0, 0, 8,
        0, 2, 2, 0, 0, 8, 3, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([8, 2, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 9, 3, 0, 1, 0,
        0, 0, 2, 0, 0, 0, 0, 2], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 4, 0, 2, 2, 2, 8, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 3, 9, 0, 0, 2, 5,
        3, 0, 0, 0, 0, 7, 0, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 7, 0, 4, 0, 0, 9, 0, 0, 0, 

input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([3, 0, 8, 0, 8, 8, 0, 4, 8, 1, 8, 8, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0,
        0, 0, 0, 0, 5, 0, 2, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([8, 4, 0, 7, 0, 0, 6, 0, 4, 0, 0, 8, 2, 8, 9, 8, 4, 0, 0, 1, 0, 0, 0, 7,
        1, 0, 8, 0, 0, 6, 3, 8], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([8, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 4, 0, 6, 0, 6, 0, 0, 0, 0, 0, 8, 1, 0,
        4, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([2, 0, 0, 0, 3, 0, 1, 2, 0, 0, 

input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([8, 2, 0, 8, 0, 0, 9, 0, 0, 4, 3, 0, 8, 0, 7, 0, 0, 0, 0, 7, 2, 0, 0, 0,
        2, 0, 9, 0, 0, 8, 0, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([1, 0, 9, 2, 0, 1, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 8, 9, 0, 8, 0, 0,
        2, 0, 0, 0, 0, 0, 1, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 8, 0, 2, 2, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([8, 7, 6, 0, 0, 0, 0, 6, 0, 8, 0, 0, 3, 0, 4, 8, 0, 0, 6, 0, 9, 0, 0, 8,
        4, 0, 8, 2, 2, 9, 9, 8], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 8, 0, 2, 0, 2, 0, 0, 1, 0, 0, 8, 9, 7, 5, 8, 8, 5, 0, 2, 0, 0, 0,
        3, 4, 0, 3, 0, 0, 0, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 1, 0, 2, 0, 0, 0, 0, 2, 8, 8, 2,
        0, 1, 9, 0, 9, 8, 9, 9], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([9, 0, 8, 8, 1, 0, 0, 8, 8, 0, 

input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 0, 8, 0, 0, 0, 0, 0, 8, 7, 0, 0, 9, 0, 1, 4, 0, 0, 2, 2, 8, 0, 2,
        2, 5, 2, 2, 8, 8, 9, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 8, 0, 0, 0, 2, 2, 1, 8, 0, 1, 8, 0, 8, 3, 6, 3, 2, 5, 0, 2, 9, 0,
        0, 4, 0, 0, 0, 0, 0, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 4, 3, 2, 2, 2, 0, 0, 2, 9, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
        0, 2, 0, 0, 0, 0, 0, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 2, 2, 0, 0, 0, 8, 8, 0, 

input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([8, 8, 0, 0, 0, 9, 8, 0, 0, 0, 2, 8, 8, 0, 1, 1, 0, 2, 0, 0, 0, 0, 8, 0,
        0, 8, 8, 2, 7, 0, 8, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 0, 0, 8, 8, 8, 8, 0, 0, 8, 4, 8, 0, 8, 0, 8, 0, 8, 2, 2, 0, 0, 8,
        0, 0, 2, 8, 0, 0, 0, 1], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([8, 8, 4, 0, 0, 9, 0, 0, 0, 0, 0, 8, 8, 8, 9, 1, 4, 8, 8, 0, 0, 8, 0, 0,
        3, 0, 8, 0, 0, 1, 0, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 

input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 4, 9, 1, 0, 0, 8, 0, 8, 0, 0, 0, 2, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 6, 0, 0, 8, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 2, 8, 8, 0, 0, 0, 0, 9, 6, 0, 0, 8, 1, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0,
        0, 1, 0, 0, 0, 0, 0, 8], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 8,
        8, 8, 0, 0, 0, 0, 9, 8], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([2, 0, 8, 0, 0, 0, 0, 0, 0, 0, 

input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 0, 0, 1, 0, 1, 0, 8, 0, 0, 1, 8, 0, 9, 0, 3, 9, 0, 8, 2, 8, 8, 1,
        2, 0, 0, 9, 0, 4, 2, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 2, 2, 2, 0, 0, 8, 0, 0, 1, 6, 0, 8, 1, 0, 0, 0, 0, 4, 2, 7, 1, 0, 0,
        7, 9, 9, 2, 0, 1, 4, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 9, 3, 9, 2, 0, 0, 9, 9, 1, 0, 2, 0, 7, 0, 8, 8, 0, 8, 0, 8, 9, 8, 0,
        0, 2, 1, 0, 0, 8, 1, 0], device='cuda:0')
input tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
output tensor([0, 0, 0, 1, 2, 0, 0, 8, 0, 0, 

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

classes = ['airplane', 'automobile','bird','cat','deer','dog','frog','horse','ship','truck']

loader = transforms.Compose([transforms.Resize((32,32)),
                             transforms.ToTensor(), 
                              #normalize
                            ])
def image_loader(image_name):

    image = Image.open(image_name)
    image = image.convert('RGB')
    image = loader(image)
    image = image.unsqueeze(0)  
    image = image.cuda()
    return image  #assumes that you're using GPU
model = model_ft
model.eval()
image = image_loader('/media/abdul/8266fdc5-61f6-47d6-9e82-aa0e56f4b319/Ilyas/Gender/CINIC/test/bird/cifar10-test-67.png')
pred = (model(image)).data.cpu().numpy().argmax()

print('Prediction: ', pred)
print('Class: ', classes[pred])


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

classes = ['airplane', 'automobile','bird','cat','deer','dog','frog','horse','ship','truck']

loader = transforms.Compose([transforms.Resize((32,32)),
                             transforms.ToTensor(), 
                              #normalize
                            ])
def image_loader(image_name):

    image = Image.open(image_name)
    image = image.convert('RGB')
    image = loader(image)
    image = image.unsqueeze(0)  
    image = image.cuda()
    return image  #assumes that you're using GPU
model = model_ft
model.eval()
image = image_loader('/media/abdul/8266fdc5-61f6-47d6-9e82-aa0e56f4b319/Ilyas/Gender/CINIC/test/horse/cifar10-test-743.png')
pred = (model(image)).data.cpu().numpy().argmax()

print('Prediction: ', pred)
print('Class: ', classes[pred])

In [ ]:


# helper function to un-normalize and display an image
def imshow(img):
    img = Image.open(img)
    plt.imshow(img)  
    
imshow('/media/abdul/8266fdc5-61f6-47d6-9e82-aa0e56f4b319/Ilyas/Gender/CINIC/test/horse/cifar10-test-743.png')


In [ ]:
import glob

images = []
for f in glob.iglob("/media/abdul/8266fdc5-61f6-47d6-9e82-aa0e56f4b319/Ilyas/Gender/CINIC/test/*/*"):
    images.append(f)
    

In [ ]:
preds = []
for image in images:
    
  img = image_loader(image)
  pred = (model(img)).data.cpu().numpy().argmax()
  preds.append(pred)    
  print('Prediction: ', pred, '  ','Class: ', classes[pred])

    
  imshow(image)  
  plt.show()